## Creating a Custom Reaction

In this tutorial, I am going to walk you through how reactions work and how to make your own custom reaction


In [ ]:
!pip install "git+https://github.com/chemgymrl/chemgymrl.git@manager"

In [ ]:
from chemistrylab.reactions.reaction_info import ReactInfo
from chemistrylab.reactions.reaction import Reaction
from chemistrylab import material,vessel

import numpy as np
from IPython.display import display,clear_output,JSON

# Adding hydronium and hydroxide materials

In [ ]:
H3O = dict(
  smiles = '[OH3+]',
  name='hydronium',
  molar_mass=19.023,
  phase='l',
  dissolves_in = {"O"},
  polarity=abs(2 * 1.24 * np.cos((109.5 / 2) * (np.pi / 180.0))),
  dissolved_components = {"[OH3+]":1},
  boiling_point_K=373.15,
  specific_heat_J_gK=4.1813,
  enthalpy_vapor_J_mol=40650.0,
  density_g_ml={'s': None, 'l': 0.997, 'g': None},
  color=0.2,
)


OH = dict(
  smiles = '[OH-]',
  name='hydroxide',
  molar_mass=17.008,
  phase='l',
  dissolves_in = {"O"},
  polarity=abs(2 * 1.24 * np.cos((109.5 / 2) * (np.pi / 180.0))),
  dissolved_components = {"[OH-]":1},
  boiling_point_K=373.15,
  specific_heat_J_gK=4.1813,
  enthalpy_vapor_J_mol=40650.0,
  density_g_ml={'s': None, 'l': 0.997, 'g': None},
  color=0.2,
)

material.register(H3O,OH)

# Setting the reactants and products

$H_2O$, $H_3O^+$, and  $OH^-$ are all both reactants and products (since we are including both the forward and reverse reaction)

In [ ]:
name="Autoionization"
REACTANTS = ["O","[OH3+]","[OH-]"]
PRODUCTS = ["O","[OH3+]","[OH-]"]
SOLVENTS = ["O"]
MATERIALS=["O","[OH3+]","[OH-]"]

# Setting rates

For each reaction we have: $k = Ae^{\frac{Ea}{RT}}$

To set this we know [$OH^-$][$H_3O^+$] = $1\cdot 10^{-14}$ at equilibrium.

Additionally we know [H_2O] is always 55.34


In [ ]:
# 55.34 is the concentration of water in water and 1e-14 is Keq in the autoionization reaction
pre_exp_arr = np.array([55.34,1e-14])*1e7
# No idea what the activation energies are
activ_energy_arr = np.array([1.0,1.0])

# Setting Stoicheometry coefficients

This will be a [reactions, reactants] shape array

In [ ]:
stoich_coeff_arr = np.array([
    [0, 1, 1], # H3O + OH -> H2O+H2O
    [1, 0, 0] # H2O + H2O -> H3O + OH
]).astype(np.float32)

# Setting concentration coefficients

This will be a [materials, reactions] shape array. It represents the change in concentrations given by each reaction. (Changes in concentration will always be within the column space of this matrix)

In [ ]:

conc_coeff_arr = np.array([
    [2, -2],
    [-1, 1],
    [-1, 1]
]).astype(np.float32)

info = ReactInfo(name,REACTANTS,PRODUCTS,SOLVENTS,MATERIALS,pre_exp_arr,activ_energy_arr,stoich_coeff_arr,conc_coeff_arr)

# Setting up the reaction

In [ ]:
reaction = Reaction(info)
v = vessel.Vessel("Water Vessel")
H2O = material.REGISTRY["O"](mol=1)
v.material_dict = {str(H2O):H2O}
v.default_dt=0.1

In [ ]:
v.get_material_dataframe()

In [ ]:
reaction.update_concentrations(v)
v.get_material_dataframe()

# Looking at the pH

In [ ]:
pH = -np.log(v.material_dict["[OH-]"].mol/v.material_dict["O"].volume_L)/np.log(10)

print(f"pH: {pH}")

# Saving as a json file

In [ ]:
info.dump_to_json("autoionization.json")
json_text = "".join(line for line in open("autoionization.json","r"))
print(json_text)

# Loading from your json file

In [ ]:
info_copy = ReactInfo.from_json("autoionization.json")

print(info_copy.name)